In [21]:
import json
import datetime
import time
import requests
import nbimporter
import random
import os
import psutil
import psycopg2
# from multiprocessing import Process, Pool
import Insert_py
import Get_py
import Tor
import Proxy_Gatherer as pg
import Init_database as indb
import email_alert as reminder
import warnings
warnings.filterwarnings("ignore")

Importing Jupyter notebook from email_alert.ipynb


In [2]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [16]:
#------- Gathering proxies for visiting teld.cn -------##
def proxy():
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
    gatherproxy = pg.ProxyGatherer(hdr)
    proxies_xici = gatherproxy.getproxies_xici()
    proxies_elite = gatherproxy.getproxies_elite()
#     proxies_usproxy = gatherproxy.getproxies_usproxy()
    # t = time.time()
#     proxiesXici_pool = gatherproxy.proxies_cleanup(proxies_xici) # Could also specify timeout limit you prefer, 
                                                                    #format--(connect time, read time)
#     proxiesElite_pool = gatherproxy.proxies_cleanup(proxies_elite)
    proxies = proxies_xici+proxies_elite
#     return proxies
    return proxies

In [4]:
def write_error(error_log, pillcode, error_msg):
    """
        Input:
            error_log - file, write error messages into designated file
    """
    error_log.write(pillcode)
    error_log.write(str(error_msg))
    error_log.write('\n')

In [5]:
def header_gen():
    import numpy as np
    app_version = np.random.randint(1,5)
    os_version = np.random.randint(0,3)+round(np.random.randint(0,3)*0.1,1)
    user_ag = "Teld/3.{a}.0 (iPhone; iOS 10.{o}; Scale/2.00)".format(a=app_version,o=os_version)
    device = "app_version=3.{a}.0&os_version=10.{o}&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=".format(a=app_version,o=os_version)
    hdr = {
        'accept': "*/*",
        'user-agent': user_ag,
        'cookie': "TELDAppID=",
        'content-length': "111",
        'connection': "keep-alive",
        'accept-encoding': "gzip, deflate",
        'device': device,
        'host': "basesg.teld.cn",
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
    }
    return hdr

In [6]:
def ChargeHist_header_gen():
    import numpy as np
    app_version = np.random.randint(1,5)
    os_version = np.random.randint(0,3)+round(np.random.randint(0,3)*0.1,1)
    user_ag = "Teld/3.{a}.0 (iPhone; iOS 10.{o}; Scale/2.00)".format(a=app_version,o=os_version)
    device = "app_version=3.{a}.0&os_version=10.{o}&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=".format(a=app_version,o=os_version)
    hdr = {
        'accept': "*/*",
        'user-agent': user_ag,
        'cookie': "TELDAppID=",
        'content-length': "250",
        'connection': "keep-alive",
        'accept-encoding': "gzip, deflate",
        'device': device,
        'host': "basesg.teld.cn",
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
    }
    return hdr
ChargeHist_hdr = ChargeHist_header_gen()

In [ ]:
def user_act_AIclean(user_act):
    pileName = str(user_act['PileName'].encode('utf-8'))
    if user_act['userNickName'] is not None:
        userNickName = str(user_act['userNickName'].encode('utf-8'))
    else:
        userNickName = None
    if user_act['carTypeName'] is not None:
        carTypeName = str(user_act['carTypeName'].encode('utf-8'))
    else:
        carTypeName = None
    startTime = user_act['chargeTime']
    #AI detect startTime
    if len(startTime) < 6:
    #if in the expression of hours:
        if hour_uni in startTime:
            idx_hour = len(startTime)-3
            hours = int(startTime[:idx_hour])
            startTime = startTime = datetime.datetime.now() - datetime.timedelta(hours=hours)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
            startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
    #if in the expression of minutes:
        if minute_uni in startTime:
            idx_min = len(startTime)-3
            minutes = int(startTime[:idx_min])
            startTime = startTime = datetime.datetime.now() - datetime.timedelta(minutes=minutes)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
            startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
    elif len(startTime) >= 6:
        startTime = '2017-' + startTime + '-07'   
    return startTime, pileName, carTypeName, userNickName

In [9]:
# ##------- Every once a while runs it update station and pile fixed info -------##
# #--------------------------------------------#
# #------- Gather all stations  -------# ****NEED TO BE FIXED 09.17.2017
# station_info = Get_py.get_allStations(hdr)
# print(len(station_info))
# for station in station_info:
#     stationId = str(station['id'])  
# #     print(stationId)
#     #------- Gather individual stations details -------#
#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)
#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)
#     #------- Gather individual pile code -------#
#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)
# #------- Gather inidividual pile details -------#
# #------- Request stationid which has user activities from db -------#
try:
    cur = conn.cursor()
    cur.execute("select distinct stationid from tbl_app_useractivity;")
    stationWithAct_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(stationWithAct_list))
# #------- Request pilecode from db for states -------#
staid_list = []
for staid in stationWithAct_list:
    staid_list.append(staid[0])
try:
    cur = conn.cursor()
    sta_list = ['2b3a35d4-cb94-4b8f-a47a-bbb054754066', '1d02640a-7f14-4fc3-baaf-60754a38fff7',
                'ac6cb946-5fff-491b-859e-55cc5f701657', 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66',
                'b134a159-16bd-4f5d-af1b-cd42599a3bab', '33a3c031-0a3a-417d-ba1b-53549bb08dc2', 
                'e4758edb-1183-4c51-887b-a52ca7abef3d', '13bdab71-110b-4448-a3c4-5c3113183546']
    #1. beijing hangkong jingmi jixie yanjiusuo chongdianzhan --'2b3a35d4-cb94-4b8f-a47a-bbb054754066',
    #2. beijing xijiao bingguan chongdianzhan -- '1d02640a-7f14-4fc3-baaf-60754a38fff7
    #3. beijing yangguanglicheng jiudian chongdianzhan --'ac6cb946-5fff-491b-859e-55cc5f701657',
    #4. beijing rongtao gongsi chongdianzhan -- '32d317a9-3d0c-4458-8357-89a2b3b1cf94'
    #5. beijing huairou qu yanshui lu -- 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'
    
    #6. shenzhen hengjiang dasha chongdianzhan -- 'b134a159-16bd-4f5d-af1b-cd42599a3bab'
    #7. shenzhen yinxing gongyeyuan chongdianzhan -- '33a3c031-0a3a-417d-ba1b-53549bb08dc2'
    #8. shenzhe wanke jiulongshan chongdianzhan -- 'e4758edb-1183-4c51-887b-a52ca7abef3d'
    #9. shenzhen jingyuan dasha chongdianzhan -- '13bdab71-110b-4448-a3c4-5c3113183546'
    stmt = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt, (tuple(sta_list),))
    
#     stmt = "select pilecode from tbl_APP_pilecode where staid = 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'"
#     cur.execute(stmt)
#     cur.execute("select pilecode from tbl_APP_pilecode")
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(pillcode_list))

# # ------- Requests stationID from db for user charging activities -------#
try:
    cur = conn.cursor()
    cur.execute("select stationid from tbl_app_eachstations;")
    station_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(station_list))

# reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15)
# i = 0
# for pile in pilecode_list:
#     #If encounter error, try one more request with a different proxy
#     try:
#         pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#     except ValueError as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     if pile_detailedInfo is not None:
#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
#     else:
#         print("Running None Func.")
#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])
#     i += 1
#     if datetime.datetime.now() >= reportTime:
#         reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15) 
#         print("Successfully run {} piles".format(i))
# #211 Running None Func.
# #71585 piles.
# #------- Request Shenzhen charging pilecode from db for states -------#
try:
    cur = conn.cursor()
#     stmt1 = "select stationid from tbl_app_eachstations where tbl_app_eachstations.location && ST_MakeEnvelope(115.37916, 39.47148, 117.15623, 40.81726, 4326);"    
    stmt1 = "select stationid from tbl_app_eachstations where tbl_app_eachstations.location && ST_MakeEnvelope(113.85955, 22.50563, 114.17335, 22.69960, 4326) intersect select distinct stationid from tbl_app_useractivity;"
    cur.execute(stmt1)
    sta_list = cur.fetchall()
    sta_list_clean = []
    for sta in sta_list:
        sta_list_clean.append(sta[0])
    stmt2 = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt2, (tuple(sta_list_clean),))
    shenzhen_pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(shenzhen_pillcode_list))

3498
86
8866


In [ ]:
# #------- Update user activity -------#
i = 0
fail = 0
hour_uni = u'\u5c0f\u65f6\u524d'
minute_uni = u'\u5206\u949f\u524d'
# proxies = proxy()
# proxy_idx = random.randint(0,len(proxies)-1) 
# os.environ["https_proxy"]=proxies[proxy_idx]['https']
# os.environ["http_proxy"]=proxies[proxy_idx]['http']
activity_dict = {}
for idx, stationID in enumerate(station_list):
    stationID = stationID[0]
#     if len(proxies) == 1: #proxy pool dried out
#         proxies = proxy()
    
#     proxy_idx = random.randint(0,len(proxies)-1)
#     os.environ["https_proxy"]=proxies[proxy_idx]['https']
#     os.environ["http_proxy"]=proxies[proxy_idx]['http']
    
    try_num = 2
#     while try_num:
#         if try_num < 2:
#             print("Retry")
#         try:
#             chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)
#         except:
#             print("Changing proxy")
#             proxies.pop(proxy_idx)
#             if len(proxies) == 1:
#                 proxies = proxy()
#                 proxy_idx = random.randint(0,len(proxies)-1)
#             else:
#                 proxy_idx = random.randint(0,len(proxies)-1)
#             os.environ["https_proxy"]=proxies[proxy_idx]['https']
#             os.environ["http_proxy"]=proxies[proxy_idx]['http']
#         try_num -= 1
#         if not try_num:
#             chargeList_detailedInfo=[]
#             fail += 1
#             print('fail +', fail, ' index', idx)
                
    try:
        chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)
    except:
        try:
#             proxies.pop(proxy_idx)
#             proxy_idx = random.randint(0,len(proxies)-1)
#             os.environ["https_proxy"]=proxies[proxy_idx]['https']
#             os.environ["http_proxy"]=proxies[proxy_idx]['http']
            chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID,ChargeHist_hdr)
        except:
#             proxies.pop(proxy_idx)
            chargeList_detailedInfo=[]
            fail += 1
            print('fail +', fail, ' index', idx)
            pass
    if len(chargeList_detailedInfo) == 0:
        i += 1
    else:
        activity_dict['{}'.format(stationID)] = chargeList_detailedInfo
print(str(i) + " stations do not have activity history.")
print(str(fail) + " connections failed attempt.")

In [15]:
for stationID in activity_dict.keys():
    chargeList_detailedInfo = activity_dict[stationID]
    for user_act in chargeList_detailedInfo:
        startTime, pileName, carTypeName, userNickName = user_act_AIclean(user_act)
        conn = Insert_py.insert_userActivity_info(conn, startTime, pileName, stationID, carTypeName, userNickName)

2903

In [7]:
def pillState_scrap(pillcode): 
    """
        Input:
            pillCode - string, each charging pile has its own pile code
        Output:
            pill_detailedInfo - dictionary of pill state details
    """
    hdr = header_gen()
    time_out = 20
    F = None
    proxies_pool = None
    #If encounter error, try one more request with a different proxy
    try:
        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
#     except ValueError as e:
#         if F is not None:
#             write_error(F, pillcode, e)
#         try:
#             pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
#         except ValueError as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
#             pill_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
# #             else:
#             pill_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         if F is not None:
#             write_error(F, pillcode, e)
#         try:
#             pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
#         except ValueError as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
#             pill_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             if F is not None:
#                 write_error(F, pillcode, e)
# #         else:
#             pill_detailedInfo = None
# #     return pill_detailedInfo, proxies_pool
    except:
        pill_detailedInfo = None
    return pill_detailedInfo

In [8]:
def insert_state(outputs):
    for pill_detailedInfo in outputs:
        if pill_detailedInfo is not None:
            db_name="db_teld"
            user_name="postgres"
            password = "postgres"
            host="127.0.0.1"
            port="5432"
            conn = indb.init_db(db_name, user_name, password, host, port)
            pillCode = str(pill_detailedInfo['pillCode'])
            stateCode = str(pill_detailedInfo['stateCode'])
            stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
            if pill_detailedInfo['stateName'] is not None:
                stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
            else:
                stateName = ''
            if pill_detailedInfo['stateNameV3'] is not None:
                stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
            else:
                stateNameV3 = ''
            conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)
            conn.close()

In [10]:
## ----- Test for single looping of multiprocessing post_request and insert -----##

# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('Before runnning script, memory use:', memoryUse, 'MB')
# # loop_num = 1
# # max_proc =10000
# pool = Pool(200)
# t = time.time()
# if __name__ == '__main__':
#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]
#     print(len(results))
#     outputs = [p.get() for p in results]
# print("success, time costs is " + str(time.time()-t)) 

# pid = os.getpid()
# py = psutil.Process(pid)
# memoryUse = py.memory_info()[0]/2.**20 #in MB
# print('After runnning script,memory use:', memoryUse, 'MB')

In [12]:
for pillcode in pillcode_list[70:]:
    print(pillcode[0])

4403060016101
4403060072102
4403060072110
4403060072112
4403060072104
4403060072106
4403060072107
4403060072113
4403060072111
4403060072103
4403060072108
4403060072109
4403060072101
4403060072115
4403060072105
4403060072114


In [11]:
# os.environ["https_proxy"]="https://165.84.167.54:8080"
# os.environ["http_proxy"]="http://165.84.167.54:8080"

In [ ]:
# pool = Pool(processes=2)
# startTime = datetime.datetime.now()
endTime = datetime.datetime.now() + datetime.timedelta(hours=168)
# proxyUpdateTime = datetime.datetime.now() + datetime.timedelta(hours=3)
# tmpproxy = {'http': 'http://101.78.219.178:8080', 'https': 'https://101.78.219.178:8080'}
# os.environ["https_proxy"]=tmpproxy['https']
# os.environ["http_proxy"]=tmpproxy['http']
# proxies = proxy()
# proxy_idx = random.randint(0,len(proxies)-1) 
# os.environ["https_proxy"]=proxies[proxy_idx]['https']
# os.environ["http_proxy"]=proxies[proxy_idx]['http']

fromaddr = "296257740.jz@gmail.com"
psw = "15710725968zzqcr"
toaddr = "hustlejanton@berkeley.edu"
subject="Courtesy Email"
while True:
    if datetime.datetime.now() >= endTime:
        body="Program terminated by time"
        reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)
        break
#     Tor.renewTor()
#     Tor.connectTor()
#     Tor.showmyIp()
    print("waking up!")
#     if datetime.datetime.now() >= proxyUpdateTime: #hit update time
#         proxies = proxy()
#         proxyUpdateTime = datetime.datetime.now() + datetime.timedelta(hours=3)
    if len(proxies) <= 1: #proxy pool dried out before next update time
        proxies = proxy()
        proxy_idx = random.randint(0,len(proxies)-1) 
        os.environ["https_proxy"]=proxies[proxy_idx]['https']
        os.environ["http_proxy"]=proxies[proxy_idx]['http']
    t = time.time()
    outputs=[]
#     results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]
#     outputs = [p.get() for p in results]
    missing_data = 0
    for pillcode in pillcode_list:
#         Tor.connectTor()
        try_num = 2
        while try_num:
            if try_num < 2:
                print("Retry")
            pill_detailedInfo = pillState_scrap(pillcode[0])
            if pill_detailedInfo is not None:
                outputs.append(pill_detailedInfo)
                break
            if pill_detailedInfo is None:
                print("Changing proxy")
                proxies.pop(proxy_idx)
                if len(proxies) == 1:
                    proxies = proxy()
                    proxy_idx = random.randint(0,len(proxies)-1)
                else:
                    proxy_idx = random.randint(0,len(proxies)-1)
                os.environ["https_proxy"]=proxies[proxy_idx]['https']
                os.environ["http_proxy"]=proxies[proxy_idx]['http']
            try_num -= 1
            if not try_num:
                missing_data += 1
    if endTime - datetime.datetime.now() <= datetime.timedelta(hours=1):
        body="Program less than one hour left."
        reminder.email_courtesy(fromaddr, toaddr, psw, subject, body)
    print("Takes "+str(time.time()-t)+" seconds")
    print("Missing {} of pills data".format(missing_data))
    insert_state(outputs)
    print("sleeping...")
    time.sleep(random.randint(600, 900))

waking up!
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca63d6cd0>, 'Connection to 201.187.110.174 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Changing proxy
Takes 253.266582012 seconds
Missing 2 of pills data
sleeping...
waking up!
('Connection aborted.', BadStatusLine("''",))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca63d6590>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retri

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca65cc210>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca65cc3d0>, 'Connection to 117.13.7.9 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retr

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca5eb8250>, 'Connection to 183.33.192.63 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca65ccd90>: Failed to establish a new connection: [Errno 111] Connection refused',)))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max r

Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Changing proxy
Retry
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
Changing proxy
Takes 468.760223866 seconds
Missing 6 of pills data
sleeping...
waking up!
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca63d6dd0>: Failed to establish a new connecti

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
failed
Changing proxy
Timeout occured
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca64d3d90>, 'Connection to 177.128.152.76 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error('Tunnel connection failed: 504 Gateway Time-out',)))
failed
Changing proxy
Retry
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App

...

waking up!
Takes 131.303336859 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 115.124670029 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 123.402879 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 127.709208965 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 115.411247969 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 115.248461962 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 117.314029217 seconds
Missing 0 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fecb015b8d0>, 'Connection to 121.43.178.58 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (

sleeping...
waking up!
Takes 155.350538969 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 161.104943991 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 160.152202845 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 154.359297037 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 156.394137859 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 172.783295155 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 160.752815008 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 161.914454937 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 164.10476613 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 166.438313961 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 169.302469969 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 163.685467958 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 160.792438984 seconds
Missing 0 of pills data
sleeping...
waking up!
Tak

HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca621bb50>, 'Connection to 114.230.30.68 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', error(104, 'Connection reset by peer')))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca60cb090>, 'Connection to 35.199.50.1 timed out. (connect timeout=20)'))
failed
Changing proxy
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SI

sleeping...
waking up!
Takes 152.096359968 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 152.114206076 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 158.381724834 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 152.422194958 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 152.063476086 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 153.568985939 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 151.593681812 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 154.082844019 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 158.530719042 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 151.305589914 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 152.474997997 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 152.22112608 seconds
Missing 0 of pills data
sleeping...
waking up!
Takes 153.356495857 seconds
Missing 0 of pills data
sleeping...
waking up!
Tak

Takes 244.591962814 seconds
Missing 1 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca5d96350>, 'Connection to 171.99.200.9 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
Takes 282.260571003 seconds
Missing 0 of pills data
sleeping...
waking up!
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7feca62842d0>, 'Connection to 112.95.207.239 timed out. (connect timeout=20)'))
failed
Changing proxy
Retry
HTTPSConnectionPool(host='basesg.teld.cn', port=443): Max retries exceeded with url: /api/invoke?SID=BaseApi-App0304_GetTerminalDetails (Caused by ProxyError('Cannot connect to proxy.', erro

In [ ]:
## pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('Before runnning script, memory use:', memoryUse, 'MB')
pool = Pool(100)
t = time.time()
if __name__ == '__main__':
    results = [pool.apply_async(insert_state, (pill_detailedInfo,)) for pill_detailedInfo in outputs]
    print(len(results))
    outputs = [p.get() for p in results]
print("success, time costs is " + str(time.time()-t)) 
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('After runnning script, memory use:', memoryUse, 'MB')